In [1]:
import pandas as pd
import featuretools as ft

In [25]:
df = pd.read_csv('dataset.csv', encoding = 'ISO-8859-1')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [ ]:
#create the customer dimension table
customer_dim = df[["Customer ID", "Customer Name", "Segment", "Country", "City", "State", "Postal Code"]].drop_duplicates(subset=['Customer ID']).reset_index(drop=True)

#create the product dimension table
product_dim = df[["Product ID", "Product Name", "Category", "Sub-Category"]].drop_duplicates(subset=['Product ID']).reset_index(drop=True)

#create the order dimension table
order_dim = df[["Order ID", "Order Date", "Ship Mode"]].drop_duplicates(subset=['Order ID']).reset_index(drop=True)

#create the Time Dimension
df['Order Date'] = pd.to_datetime(df['Order Date'])
time_dim = df[['Order Date']].drop_duplicates()
time_dim['Day'] = time_dim['Order Date'].dt.day
time_dim['Month'] = time_dim['Order Date'].dt.month
time_dim['Quarter'] = time_dim['Order Date'].dt.quarter
time_dim['Year'] = time_dim['Order Date'].dt.year
time_dim.rename(columns={'Order Date': 'Date'}, inplace=True)
time_dim.drop_duplicates(subset=['Date']).reset_index(drop=True, inplace=True)

# 5. Create the Fact Table
sales_fact = df[[
    "Order ID", "Product ID", "Customer ID", "Order Date", "Region",
    "Sales", "Quantity", "Discount", "Profit"
]].copy()

sales_fact['Sale ID'] = sales_fact.index
sales_fact.rename(columns={'Order Date': 'Date'}, inplace=True)
sales_fact = sales_fact[['Sale ID', 'Order ID', 'Product ID', 'Customer ID', 'Date', 'Region', 'Sales', 'Quantity', 'Discount', 'Profit']]


In [42]:
# 5. Create the Fact Table
sales_fact = df[[
    "Order ID", "Product ID", "Customer ID", "Order Date", "Region",
    "Sales", "Quantity", "Discount", "Profit"
]].copy()

sales_fact['Sale ID'] = sales_fact.index
sales_fact.rename(columns={'Order Date': 'Date'}, inplace=True)
sales_fact = sales_fact[['Sale ID', 'Order ID', 'Product ID', 'Customer ID', 'Date', 'Region', 'Sales', 'Quantity', 'Discount', 'Profit']]


In [43]:
# Save the tables to CSV files
customer_dim.to_csv("dim_customers.csv", index=False)
product_dim.to_csv("dim_products.csv", index=False)
order_dim.to_csv("dim_orders.csv", index=False)
time_dim.to_csv("dim_time.csv", index=False)
sales_fact.to_csv("fact_sales.csv", index=False)

In [44]:
sales_fact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Sale ID      9994 non-null   int64         
 1   Order ID     9994 non-null   object        
 2   Product ID   9994 non-null   object        
 3   Customer ID  9994 non-null   object        
 4   Date         9994 non-null   datetime64[ns]
 5   Region       9994 non-null   object        
 6   Sales        9994 non-null   float64       
 7   Quantity     9994 non-null   int64         
 8   Discount     9994 non-null   float64       
 9   Profit       9994 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(4)
memory usage: 780.9+ KB


In [45]:
time_dim.head()

,Date,Day,Month,Quarter,Year
0,2016-11-08,8,11,4,2016
2,2016-06-12,12,6,2,2016
3,2015-10-11,11,10,4,2015
5,2014-06-09,9,6,2,2014
12,2017-04-15,15,4,2,2017


# Deep Feature Synthesis

In [ ]:
es = ft.EntitySet(id="ecommerce")

es = es.add_dataframe(dataframe_name="customers", dataframe=customer_dim, index="Customer ID")
es = es.add_dataframe(dataframe_name="products", dataframe=product_dim, index="Product ID")
es = es.add_dataframe(dataframe_name="orders", dataframe=order_dim, index="Order ID")
es = es.add_dataframe(dataframe_name="time", dataframe=time_dim, index="Date")
es = es.add_dataframe(dataframe_name="sales", dataframe=sales_fact, index='Sale ID')  

c:\Users\ProUser\anaconda3\Lib\site-packages\featuretools\entityset\entityset.py:724: UserWarning: A Woodwork-initialized DataFrame was provided, so the following parameters were ignored: index
  warnings.warn(


In [48]:
relationships = [
    ("customers", "Customer ID", "sales", "Customer ID"),
    ("products", "Product ID", "sales", "Product ID"),
    ("orders", "Order ID", "sales", "Order ID"),
    ("time", "Date", "sales", "Date"),
]

es.add_relationships(relationships)

c:\Users\ProUser\anaconda3\Lib\site-packages\featuretools\entityset\entityset.py:393: UserWarning: Logical type Categorical for child column Customer ID does not match parent column Customer ID logical type Unknown. Changing child logical type to match parent.
  warnings.warn(
c:\Users\ProUser\anaconda3\Lib\site-packages\featuretools\entityset\entityset.py:393: UserWarning: Logical type Categorical for child column Product ID does not match parent column Product ID logical type Unknown. Changing child logical type to match parent.
  warnings.warn(


Entityset: ecommerce
  DataFrames:
    customers [Rows: 793, Columns: 7]
    products [Rows: 1862, Columns: 4]
    orders [Rows: 5009, Columns: 3]
    time [Rows: 1237, Columns: 5]
    sales [Rows: 9994, Columns: 10]
  Relationships:
    sales.Customer ID -> customers.Customer ID
    sales.Product ID -> products.Product ID
    sales.Order ID -> orders.Order ID
    sales.Date -> time.Date

In [49]:
# Perform feature synthesis for the target entity (order_details)
features, feature_names = ft.dfs(
    entityset=es,
    target_dataframe_name="sales",
    max_depth=2,
    verbose = True
)

Built 138 features
Elapsed: 00:00 | Progress:   0%|          

c:\Users\ProUser\anaconda3\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function min at 0x0000021893F1D1C0> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  ).agg(to_agg)
c:\Users\ProUser\anaconda3\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function std at 0x0000021893F1DBC0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  ).agg(to_agg)
c:\Users\ProUser\anaconda3\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function sum at 0x0000021893F1C9A0> is currently using SeriesGroupBy.sum. In a future version of pandas, t

Elapsed: 00:00 | Progress:  12%|█▏        

c:\Users\ProUser\anaconda3\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function min at 0x0000021893F1D1C0> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  ).agg(to_agg)
c:\Users\ProUser\anaconda3\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function std at 0x0000021893F1DBC0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  ).agg(to_agg)
c:\Users\ProUser\anaconda3\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function max at 0x0000021893F1D080> is currently using SeriesGroupBy.max. In a future version of pandas, t

Elapsed: 00:00 | Progress:  24%|██▍       

c:\Users\ProUser\anaconda3\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function std at 0x0000021893F1DBC0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  ).agg(to_agg)
c:\Users\ProUser\anaconda3\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function mean at 0x0000021893F1DA80> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ).agg(to_agg)
c:\Users\ProUser\anaconda3\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function sum at 0x0000021893F1C9A0> is currently using SeriesGroupBy.sum. In a future version of pandas

Elapsed: 00:02 | Progress:  37%|███▋      

c:\Users\ProUser\anaconda3\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function mean at 0x0000021893F1DA80> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ).agg(to_agg)
c:\Users\ProUser\anaconda3\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function max at 0x0000021893F1D080> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ).agg(to_agg)
c:\Users\ProUser\anaconda3\Lib\site-packages\featuretools\computational_backends\feature_set_calculator.py:785: FutureWarning: The provided callable <function min at 0x0000021893F1D1C0> is currently using SeriesGroupBy.min. In a future version of pandas

Elapsed: 00:02 | Progress: 100%|██████████


In [50]:
feature_index = []
feature_names_list = []
feature_descriptions_list = []

for index, feature in enumerate(feature_names):
    feature_name = feature.get_name()
    feature_description = ft.describe_feature(feature)
    feature_index.append(index)
    feature_names_list.append(feature_name)
    feature_descriptions_list.append(feature_description)

feature_des = pd.DataFrame({
    'Feature Index': feature_index,
    'Feature Name': feature_names_list,
    'Description': feature_descriptions_list,
})

features.to_csv('order_details features.csv', index=False)
feature_des.to_csv('order details feature data dictionary.csv', index=False)

In [51]:
features.head()

,Region,Sales,Quantity,Discount,Profit,DAY(Date),MONTH(Date),WEEKDAY(Date),YEAR(Date),customers.Segment,...,time.STD(sales.Quantity),time.STD(sales.Sales),time.SUM(sales.Discount),time.SUM(sales.Profit),time.SUM(sales.Quantity),time.SUM(sales.Sales),time.DAY(Date),time.MONTH(Date),time.WEEKDAY(Date),time.YEAR(Date)
Sale ID,,,,,,,,,,,,,,,,,,,,,
0,South,261.9600,2,0.00,41.9136,8,11,1,2016,Consumer,...,0.707107,332.326045,0.00,261.4956,5.0,993.9000,8,11,1,2016
1,South,731.9400,3,0.00,219.5820,8,11,1,2016,Consumer,...,0.707107,332.326045,0.00,261.4956,5.0,993.9000,8,11,1,2016
2,West,14.6200,2,0.00,6.8714,12,6,6,2016,Corporate,...,2.453022,221.599424,3.80,282.5019,94.0,2962.5630,12,6,6,2016
3,South,957.5775,5,0.45,-383.0310,11,10,6,2015,Consumer,...,2.081666,537.198736,0.65,-378.2781,8.0,1011.8955,11,10,6,2015
4,South,22.3680,2,0.20,2.5164,11,10,6,2015,Consumer,...,2.081666,537.198736,0.65,-378.2781,8.0,1011.8955,11,10,6,2015


In [52]:
feature_names

[<Feature: Region>,
 <Feature: Sales>,
 <Feature: Quantity>,
 <Feature: Discount>,
 <Feature: Profit>,
 <Feature: DAY(Date)>,
 <Feature: MONTH(Date)>,
 <Feature: WEEKDAY(Date)>,
 <Feature: YEAR(Date)>,
 <Feature: customers.Segment>,
 <Feature: customers.Country>,
 <Feature: customers.State>,
 <Feature: customers.Postal Code>,
 <Feature: products.Category>,
 <Feature: products.Sub-Category>,
 <Feature: orders.Ship Mode>,
 <Feature: time.Day>,
 <Feature: time.Month>,
 <Feature: time.Quarter>,
 <Feature: time.Year>,
 <Feature: customers.COUNT(sales)>,
 <Feature: customers.MAX(sales.Discount)>,
 <Feature: customers.MAX(sales.Profit)>,
 <Feature: customers.MAX(sales.Quantity)>,
 <Feature: customers.MAX(sales.Sales)>,
 <Feature: customers.MEAN(sales.Discount)>,
 <Feature: customers.MEAN(sales.Profit)>,
 <Feature: customers.MEAN(sales.Quantity)>,
 <Feature: customers.MEAN(sales.Sales)>,
 <Feature: customers.MIN(sales.Discount)>,
 <Feature: customers.MIN(sales.Profit)>,
 <Feature: customers.MI